## Импорты

In [7]:
import pandas as pd
import sqlite3

## Подключаемся к БД и читаем схему таблицы

In [9]:
conn = sqlite3.connect("/content/checking-logs-RES_ex02.sqlite")

schema = pd.read_sql("PRAGMA table_info(test);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [12]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,pageviews
1,checker
2,deadlines
3,datamart
4,test
5,control


## Читаем 10 стр БД

In [15]:
preview = pd.read_sql("SELECT * FROM test LIMIT 10;", conn)
preview

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [17]:
preview = pd.read_sql("SELECT * FROM deadlines LIMIT 10;", conn)
preview

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


## Минимальный дельта (часов) до дедлайна

In [19]:
query_min = '''
SELECT
    t.uid,
    MIN((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
ORDER BY diff_hours ASC
LIMIT 1
'''
df_min = pd.read_sql(query_min, conn)
df_min

,uid,diff_hours
0,user_25,2.867236


## Максимальный дельта (часов) до дедлайна

In [21]:
query_max = '''
SELECT
    t.uid,
    MAX((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
ORDER BY diff_hours ASC
LIMIT 1
'''
df_max = pd.read_sql(query_max, conn)
df_max

,uid,diff_hours
0,user_18,10.973376


## Средний дельта (часов) до дедлайна

In [22]:
query_avg = '''
SELECT AVG((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
'''
df_avg = pd.read_sql(query_avg, conn)
df_avg

,avg_diff_hours
0,89.687686


## Создание таблицы views_diff

In [23]:
query_views_diff = '''
SELECT
    t.uid,
    AVG((julianday(datetime(d.deadlines, 'unixepoch')) - julianday(t.first_commit_ts)) * 24) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test t
JOIN deadlines d ON t.labname = d.labs
LEFT JOIN pageviews p ON t.uid = p.uid
WHERE t.labname != 'project1'
GROUP BY t.uid
'''
views_diff = pd.read_sql(query_views_diff, conn)
views_diff.head()

,uid,avg_diff,pageviews
0,user_1,65.119644,140
1,user_10,75.242310,445
2,user_14,159.568696,429
3,user_17,62.207514,235
4,user_18,6.367907,9


## Расчёт корреляции

In [24]:
correlation = views_diff["avg_diff"].corr(views_diff["pageviews"])
correlation

np.float64(0.18504199382651879)

## Закрываем соединение с БД

In [25]:
conn.close()